In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import warnings
warnings.filterwarnings("ignore")

In [2]:
mnist = input_data.read_data_sets('MNIST.data', one_hot=True)

batch_size = 100
n_batch = mnist.train.num_examples // batch_size
n_test_batch = mnist.train.num_examples // batch_size

#只要是用于权值，偏置
def variable_summaries(var):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean',mean)
    with tf.name_scope('stddev'):
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
    tf.summary.scalar('stddev',stddev)
    tf.summary.scalar('max',tf.reduce_max(var))
    tf.summary.scalar('min',tf.reduce_min(var))
    tf.summary.histogram('histogrom',var)   #直方图


def weight_variable(shape,name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial,name=name)

def bias_variable(shape,name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial,name=name)

def conv2d(x, W):
    #x 有4维。[batch, 长, 宽, 深度]
    #W 过滤器4维。 [长, 宽, 输入深度, 输出深度]
    #tf.nn.conv2d同样传入4个参数 conv2d(x,w,strides=[4个参数], padding='SAME)  strides步长。只涉及中间两个参数，左右/上下走
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize=[1,x,y,1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST.data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST.data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST.data\t10k-images-idx3-ubyte.gz
Extracting MNIST.data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
with tf.name_scope('input'):
    #定义两个placeholder
    x = tf.placeholder(tf.float32, [None,784], name='x-input')
    y = tf.placeholder(tf.float32, [None,10], name='y-output')
    with tf.name_scope('x_image'):
        x_image = tf.reshape(x, [-1,28,28,1],name='x-image')
        
with tf.name_scope('Conv1'):
    with tf.name_scope('W_conv1'):
        W_conv1 = weight_variable([5,5,1,32], name='W_conv1')
    with tf.name_scope('b_conv1'):
        b_conv1 = bias_variable([32], name='b_conv1')
    
    #池化层一般不算单独一层，放在卷积层理
    with tf.name_scope('conv2d_1'):
        conv2d_1 = conv2d(x_image, W_conv1) + b_conv1
    with tf.name_scope('relu'):
        h_conv1 = tf.nn.relu(conv2d_1)
    with tf.name_scope('h_pool1'):
        h_pool1 = max_pool_2x2(h_conv1)
        
with tf.name_scope('Conv2'):
    with tf.name_scope('W_conv2'):
        W_conv2 = weight_variable([5,5,32,64],name='W_conv2')
    with tf.name_scope('b_conv2'):
        b_conv2 = bias_variable([64], name='b_conv2')
    
    with tf.name_scope('conv2d_2'):
        conv2d_2 = conv2d(h_pool1, W_conv2) + b_conv2
    with tf.name_scope('relu'):
        h_conv2 = tf.nn.relu(conv2d_2)
    with tf.name_scope('h_pool2'):
        h_pool2 = max_pool_2x2(h_conv2)
        
with tf.name_scope('fc1'):
    with tf.name_scope('W_fc1'):
        W_fc1 = weight_variable([7*7*64, 1024], name='W_fc1')
    with tf.name_scope('b_fc1'):
        b_fc1 = bias_variable([1024], name='b_fc1')
    
    with tf.name_scope('h_pool2_flat'):
        h_pool_flat = tf.reshape(h_pool2, [-1,7*7*64],name='h_pool_flat')
    
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(h_pool_flat,W_fc1) + b_fc1
    with tf.name_scope('relu'):
        h_fc1 = tf.nn.relu(wx_plus_b)
    
    #dropout
    with tf.name_scope('keep_prob'):
        keep_prob = tf.placeholder(tf.float32,name='keep_prob')
    with tf.name_scope('h_fc1_drop'):
        h_hc1_drop = tf.nn.dropout(h_fc1, keep_prob, name='h_fc1_drop')
        
with tf.name_scope('fc2'):
    with tf.name_scope('W_fc2'):
        W_fc2 = weight_variable([1024, 10], name='W_fc2')
    with tf.name_scope('b_fc2'):
        b_fc2 = bias_variable([10], name='b_fc2')    
    
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(h_hc1_drop,W_fc2) + b_fc2
    with tf.name_scope('relu'):
        h_fc2 = tf.nn.relu(wx_plus_b)
    
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(h_fc2)
        
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
    #显示标量信息。用来可视化
    tf.summary.scalar('cross_entropy',cross_entropy)
    
with tf.name_scope('train_step'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    with tf.name_scope('accuracy_rate'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        
        #
        tf.summary.scalar('accuracy',accuracy)  #没必要使用variable_summaries.上面的权重有很多。可以看下均值等。这个一次就只有一个

        
        
#合并所有的merge
merged = tf.summary.merge_all()  #合并上面所有的summary



with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('loss/train',sess.graph)
    test_writer = tf.summary.FileWriter('loss/test',sess.graph)
    for i in range(1001):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        
        sess.run(train_step,feed_dict={x:batch_xs, y:batch_ys,keep_prob:0.5})
        
        #记录训练集计算的参数
        summary = sess.run(merged, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
                
        #上面两步可以合并写
        #summary,_ = summary = sess.run([merged,train_step], feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
        
        #写到train_writer和test_writer中。  tf.summary.scalar会记录每一个值。 写到磁盘中则使用 tf.summary.FileWriter     
        train_writer.add_summary(summary,i)
        
        batch_xs,batch_ys = mnist.test.next_batch(batch_size)
        
        #记录训练集计算的参数
        summary = sess.run(merged, feed_dict={x:batch_xs, y:batch_ys, keep_prob:1.0})
        
        test_writer.add_summary(summary,i)
        
        test_acc = 0
        if i%100 == 0:
            
            for batch in range(n_test_batch):
                batch_test_xs,batch_test_ys = mnist.test.next_batch(batch_size)
                test_acc += sess.run(accuracy, feed_dict={x: batch_test_xs, y: batch_test_ys,keep_prob:1})
            
            print("Iter "+ str(i) + ",Testing Accuracy " + str(test_acc/(batch+1))) 

Iter 0,Testing Accuracy 0.10123636345971715
Iter 100,Testing Accuracy 0.4490181796659123
Iter 200,Testing Accuracy 0.7181090932542628
Iter 300,Testing Accuracy 0.827818180214275
Iter 400,Testing Accuracy 0.8426909081502394
Iter 500,Testing Accuracy 0.847490906607021
Iter 600,Testing Accuracy 0.8490363630381498
Iter 700,Testing Accuracy 0.8610909075086767
Iter 800,Testing Accuracy 0.9440000011704185
Iter 900,Testing Accuracy 0.9557818207957528
Iter 1000,Testing Accuracy 0.9568000028350137


In [ ]:
#笔记
#    当cpu和GPU版本的tensorflow同时安装时。需要使用Anaconda的GPU的cmd.运行tensorboard。
#    tensorboard==1.13.1可能报错。回滚安装1.12.2 

#   变量管理的优点：
#          ①当层数变多时，变量管理方便阅读
#          ②可视化时，层次非常明显
#          ③使用训练好的神经网络进行推导时，可以直接调用